[current dir](.)

In [1]:
from collections import defaultdict
from copy import copy
import heapq

filename = 'inputs/day20.txt'
# filename = 'inputs/day20-test1.txt'
# filename = 'inputs/day20-test2.txt'
# filename = 'inputs/day20-test3.txt'

with open(filename) as file:
    inputstr = file.read().splitlines()
input = [list(n) for n in inputstr]

In [2]:
[l[:5] for l in input[:5]]
# [str(l) for l in input]

[[' ', ' ', ' ', ' ', ' '],
 [' ', ' ', ' ', ' ', ' '],
 [' ', ' ', '#', '#', '#'],
 [' ', ' ', '#', '.', '#'],
 [' ', ' ', '#', '.', '#']]

In [3]:
class Point:
    def __init__(self,x,y):
        self.x = x
        self.y = y
    def __add__(self, other): 
        return Point(self.x + other.x, self.y + other.y)
    def __repr__(self):
        return '(' + str(self.x) + "," + str(self.y) + ')'
    def __eq__(self, other):
        return self.x == other.x and self.y == other.y
    def __hash__(self):
        return hash((self.x, self.y))

def portal(pt):
    if 0 <= pt.y-1 < len(input) and 0 <= pt.x < len(input[pt.y-1]) and input[pt.y-1][pt.x] == '.':
        return (input[pt.y][pt.x] + input[pt.y+1][pt.x], Point(pt.x, pt.y-1))
    if 0 <= pt.y+1 < len(input) and 0 <= pt.x < len(input[pt.y+1]) and input[pt.y+1][pt.x] == '.':
        return (input[pt.y-1][pt.x] + input[pt.y][pt.x], Point(pt.x, pt.y+1))
    if 0 <= pt.y < len(input) and 0 <= pt.x-1 < len(input[pt.y]) and input[pt.y][pt.x-1] == '.':
        return (input[pt.y][pt.x] + input[pt.y][pt.x+1], Point(pt.x-1, pt.y))
    if 0 <= pt.y < len(input) and 0 <= pt.x+1 < len(input[pt.y]) and input[pt.y][pt.x+1] == '.':
        return (input[pt.y][pt.x-1] + input[pt.y][pt.x], Point(pt.x+1, pt.y))
    return None

def is_inside(portal_pt):
    return 3 <= portal_pt.y <= len(input)-3 and 3 <= portal_pt.x <= len(input[0])-3
portals = {}
for y in range(len(input)):
    for x,c in enumerate(input[y]):
        pt = Point(x,y)
        if c.isalpha() and portal(pt) != None:
            (pname, ppoint) = portal(pt)
            inside = is_inside(pt)
            portals[pname, inside]= (ppoint, Point(x,y))
print("portals",portals)

portal_translation = {}
for ((portal, inside), (dpoint, ppoint)) in [((p,i),pos) for ((p,i), pos) in portals.items() if p not in ['AA','ZZ']]:
    other_end = portals[portal, not inside]
    portal_translation[ppoint] = (other_end[0], other_end[1], portal)
print("portal_translation",portal_translation)

aa = portals['AA', False]
zz = portals['ZZ', False]

directions = [Point(0,-1), Point(0,1), Point(-1, 0), Point(1, 0)]
distances = defaultdict(list)
for (pname, inside), (dpoint, ppoint) in portals.items():
    queue = [(ppoint, dpoint)]
    distance = 0
    while queue:
        next = []
        for (origin, position) in queue:
            for delta in [d for d in directions if position + d != origin]:
                neighbor = position + delta
                if input[neighbor.y][neighbor.x] == '.':
                    next.append((position, neighbor))
                elif input[neighbor.y][neighbor.x].isalpha() and neighbor not in [aa[1], zz[1]]:
                    (nposition, norigin, nname) = portal_translation[neighbor]
                    distances[(pname, is_inside(ppoint))].append((nname, is_inside(neighbor), norigin, distance))
                elif neighbor == zz[1]:
                    distances[(pname, is_inside(ppoint))].append(('ZZ', is_inside(neighbor), norigin, distance))
                elif neighbor == aa[1]:
                    distances[(pname, is_inside(ppoint))].append(('AA', is_inside(neighbor), norigin, distance))
        queue = next
        distance += 1
print("distances",distances)

portals {('SI', False): ((39,2), (39,1)), ('ZH', False): ((47,2), (47,1)), ('JS', False): ((55,2), (55,1)), ('XC', False): ((63,2), (63,1)), ('QT', False): ((67,2), (67,1)), ('XF', False): ((71,2), (71,1)), ('LE', False): ((77,2), (77,1)), ('ZZ', False): ((81,2), (81,1)), ('WN', False): ((87,2), (87,1)), ('DV', True): ((43,36), (43,37)), ('OV', True): ((47,36), (47,37)), ('FZ', True): ((61,36), (61,37)), ('XF', True): ((63,36), (63,37)), ('PY', True): ((65,36), (65,37)), ('WX', True): ((71,36), (71,37)), ('GS', True): ((79,36), (79,37)), ('RV', True): ((85,36), (85,37)), ('JX', True): ((36,41), (37,41)), ('PY', False): ((2,43), (1,43)), ('PP', True): ((92,45), (91,45)), ('OF', False): ((126,47), (127,47)), ('OV', False): ((2,51), (1,51)), ('ES', True): ((36,53), (37,53)), ('DV', False): ((126,53), (127,53)), ('ZJ', True): ((92,55), (91,55)), ('BL', True): ((36,61), (37,61)), ('GM', False): ((126,61), (127,61)), ('FZ', False): ((2,63), (1,63)), ('TW', True): ((92,63), (91,63)), ('JX', F

Part 1

In [4]:
import heapq

def part1():
    queue = [(0, 'AA', False)]
    while queue:
        (pdist, pname, pinside) = heapq.heappop(queue)
        if pname == 'ZZ':
            return pdist - 1
        possible_portals = [(n,i,p,d) for (n,i,p,d) in distances[(pname, pinside)]]
        for nname, ninside, npos, ndist in possible_portals:
            heapq.heappush(queue, (pdist+ndist+1, nname, not ninside))

part1()

684

Part 2

In [5]:
import heapq

def part2():
    queue = [(0, 'AA', False, 0)]
    while queue:
        (pdist, pname, pinside, level) = heapq.heappop(queue)
        possible_portals = [(n,i,p,d) for (n,i,p,d) in distances[(pname, pinside)]]
        for nname, ninside, npos, ndist in possible_portals:
            if nname == 'ZZ' and level == 0:
                return pdist+ndist
            level_next = level + (1 if ninside else -1)
            if level_next >= 0:
                heapq.heappush(queue, (pdist+ndist+1, nname, not ninside,level_next))

part2()

7758